In [8]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from sklearn.linear_model import LinearRegression
from nba_api.stats.static import players
import pickle
import unidecode
from bs4 import BeautifulSoup
import requests
import time
import matplotlib.pyplot as plt
import datetime as dt
import re

today = dt.date.today()
today_month = today.month
if (today_month > 7):
    current_year = today.year + 1
else:
    current_year = today.year

In [28]:
team_map_results = {
    'LAL' : 'Los Angeles Lakers',
    'LAC' : 'Los Angeles Clippers',
    'POR' : 'Portland Trailblazers',
    'CLE' : 'Cleveland Cavaliers',
    'DEN' : 'Denver Nuggets',
    'DAL' : 'Dallas Mavericks',
    'UTA' : 'Utah Jazz',
    'SA' : 'San Antonio Spurs',
    'ATL' : 'Atlanta Hawks',
    'CHA' : 'Charlotte Hornets',
    'CHI' : 'Chicago Bulls',
    'DET' : 'Detroit Pistons',
    'MIL' : 'Milwaukee Bucks',
    'ORL' : 'Orlando Magic',
    'MIN' : 'Minnesota Timberwolves',
    'PHO' : 'Phoenix Suns',
    'BOS' : 'Boston Celtics',
    'IND' : 'Indiana Pacers',
    'SAC' : 'Sacramento Kings',
    'TOR' : 'Toronto Raptors',
    'WAS' : 'Washington Wizards',
    'BKN' : 'Brooklyn Nets',
    'NO' : 'New Orleans Pelicans',
    'DAL' : 'Dallas Mavericks',
    'PHI' : 'Philadelphia 76ers',
    'MIA' : 'Miami Heat',
    'MEM' : 'Memphis Grizzlies',
    'GS' : 'Golden State Warriors',
    'NY' : 'New York Knicks',
    'HOU' : 'Houston Rockets',
    'OKC' : 'Oklahoma City Thunder'
}

In [23]:
# Instantiating selenium driver
link = 'https://www.cbssports.com/nba/schedule/'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(link)



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST driver version for 98.0.4758
Driver [/Users/hsinger24/.wdm/drivers/chromedriver/mac64/98.0.4758.80/chromedriver] found in cache


In [24]:
# Getting table for yesterday's games and formatting
day_buttons = driver.find_elements_by_class_name("ToggleContainer-text")
yesterday = day_buttons[0]
yesterday.click()
WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//*[@id='TableBase']/div/div/table")))
html_yesterday = driver.page_source
tables_yesterday = pd.read_html(html_yesterday)
try:
    yesterday_schedule = tables_yesterday[1]
except:
    yesterday_schedule = tables_yesterday[0]
yesterday_schedule = yesterday_schedule[['Away', 'Home', 'Result']]

# Populating yesterday's schedule w/ winner
score_regex = '\d{2,3}'
test_subject = yesterday_schedule.iloc[0, 2]
yesterday_schedule['Winner'] = ''
for index, row in yesterday_schedule.iterrows():
    winner = row.Result[:3]
    yesterday_schedule.loc[index, 'Winner'] = winner
yesterday_schedule['Winner'] = yesterday_schedule.Winner.apply(lambda x: team_map_results[x])
yesterday_schedule